The New York Neighborhood Recommender

I. Purpose

Just imagine finding a well payed job in New York. Finally having this grand and diverse city as yo
ur new home. As many people found out, it can also be tricky to go there. What if you feel strange? Or you find out you don't like your Neighborhood? This App will help you getting along by finding the right Neighborhood to move to based on where you live now. So you won't feel alone and strange. For anyone renting apartments it can be profitable as well. People find the right place to call home faster and are more happy, thus they will stay longer. Plus they will recommend you.

II. Data Acquisition

We will use data based on the Foursquare API. With it we can gather information on all revenues in a given Neighborhood of New Yor. We will also gather the same information for the place you are currently living in.

Lets start! First with the New York Data. We downloaded a dataset with all Neighborhoods and geolocations.

In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [9]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Wakefield,40.894705,-73.847201
1,Co-op City,40.874294,-73.829939
2,Eastchester,40.887556,-73.827806
3,Fieldston,40.895437,-73.905643
4,Riverdale,40.890834,-73.912585


Now get the hottest venues.

In [13]:
CLIENT_ID = 'I5BFYOW54HP3LSQ4EVOELKFQE5HHLKVVBVZPJL3P2T1CVCSQ' # your Foursquare ID
CLIENT_SECRET = 'Z35ONVJSALJPPKTLRNHKF34E5B4CNVG4UNF04VFDK1OV2VDE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

Your credentails:
CLIENT_ID: I5BFYOW54HP3LSQ4EVOELKFQE5HHLKVVBVZPJL3P2T1CVCSQ
CLIENT_SECRET:Z35ONVJSALJPPKTLRNHKF34E5B4CNVG4UNF04VFDK1OV2VDE


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue',                   
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )
newyork_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Dunkin Donuts,40.890631,-73.849027,Donut Shop
4,Wakefield,40.894705,-73.847201,SUBWAY,40.890656,-73.849192,Sandwich Place


In [18]:
# one hot encoding
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]

newyork_onehot.head()

newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()


In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']

for ind in np.arange(newyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Supermarket,Chinese Restaurant,Spa,Bakery,Breakfast Spot,Fried Chicken Joint,Spanish Restaurant,Fast Food Restaurant,Smoke Shop
1,Annadale,American Restaurant,Diner,Bakery,Restaurant,Sports Bar,Pizza Place,Pet Store,Pharmacy,Train Station,Women's Store
2,Arden Heights,Pharmacy,Pizza Place,Coffee Shop,Bus Stop,Women's Store,Field,Event Space,Exhibit,Eye Doctor,Factory
3,Arlington,Bus Stop,Food Service,American Restaurant,Grocery Store,Fish Market,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market
4,Arrochar,Bus Stop,Italian Restaurant,Deli / Bodega,Supermarket,Hotel,Middle Eastern Restaurant,Bagel Shop,Pizza Place,Sandwich Place,Polish Restaurant
5,Arverne,Surf Spot,Metro Station,Pizza Place,Bed & Breakfast,Playground,Bus Stop,Board Shop,Sandwich Place,Donut Shop,Thai Restaurant
6,Astoria,Middle Eastern Restaurant,Bar,Bakery,Hookah Bar,Greek Restaurant,Seafood Restaurant,Italian Restaurant,Pub,Ice Cream Shop,Café
7,Astoria Heights,Italian Restaurant,Hostel,Deli / Bodega,Plaza,Playground,Pizza Place,Cocktail Bar,Bus Station,Burger Joint,Bowling Alley
8,Auburndale,Athletics & Sports,Mobile Phone Shop,Train Station,Donut Shop,Fast Food Restaurant,Toy / Game Store,Miscellaneous Shop,Pharmacy,Discount Store,Pet Store
9,Bath Beach,Pharmacy,Donut Shop,Kids Store,Sushi Restaurant,Italian Restaurant,Bubble Tea Shop,Fast Food Restaurant,Chinese Restaurant,Women's Store,German Restaurant


Now the data for your location. I chose my home as an example. Later in the analysis we can see which New York Neighborhoods are similar bases on the data collected.

In [10]:
home_name = 'Eimsbüttel'
home_lat = '53.555464'
home_lon = '9.811365'

neighborhoods = neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)